In [1]:
# interactive plot to check DVN results
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from plotly.subplots import make_subplots
import pandas as pd
import time
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, SpectralEmbedding
import plotly_express as px
import plotly.graph_objs as go
import ipywidgets as widgets
import os
from sklearn import preprocessing
from ast import literal_eval
min_max_scaler = preprocessing.MinMaxScaler()
n_neighbors = 10

class interactive_graph():
    def __init__(self,df,grouping_by="label",title=""):
        p = px.scatter(df, x="v1",y="v2",color=grouping_by,hover_data=["path"],color_discrete_sequence=px.colors.qualitative.Dark24)
    
        for trace in p.data:
            trace.update(hoverinfo="none",hovertemplate= '')
        
        self.hover_data = widgets.Textarea()  
        self.aud=widgets.Audio(autoplay=True,loop=False,embedding=True)

        layout = go.Layout(hovermode=False,)
        self.fig  = go.FigureWidget(p)
        self.fig.update_layout(
            title={
                'text': title,
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})
        for f in self.fig.data:
            f.on_hover(self.hover_fn)
            
    def hover_fn(self,trace, points, state):
        if points.point_inds:
            ind = points.point_inds[0]
            drmName=trace.customdata[ind][0][2:]
            filename=os.getcwd()+"/"+drmName
            with open(filename,'rb') as f:
                audio_data = f.read()
            self.aud.value=audio_data
            self.hover_data.value = str(drmName)+"\n"

def plotly_able_df(df):
    
    df["feats"]=df["feats"].apply(literal_eval)
    X=df["feats"]
    #convert series of arrays into a numpy array
    X=pd.DataFrame(X.to_list()).to_numpy()
    X = min_max_scaler.fit_transform(X)
    time_start = time.time()
    tsne = TSNE(n_components=2, random_state=0, perplexity=80, verbose=1)
    # tsne = Isomap(n_neighbors, n_components=2)
    # tsne = LocallyLinearEmbedding(n_neighbors,n_components=4)
    # tsne = SpectralEmbedding(n_neighbors=10,n_components=5,random_state=1)
    X_2d = tsne.fit_transform(X)
    df2=pd.concat([df["path"],df["label"],df["drum_type"],pd.Series(X_2d[:,0]),pd.Series(X_2d[:,1])],axis=1)
    df2.columns=["path","label","drum_type","v1","v2"]
    df2.label = df2.label.astype('str')
    return df2

df_64=plotly_able_df(pd.read_csv("feature_extraction/csvs/spec_encode_64_290_0.002042.pt.csv"))
df_8=plotly_able_df(pd.read_csv("feature_extraction/csvs/spec_encode_sm_8_270_0.003504.pt.csv"))

FileNotFoundError: [Errno 2] File b'feature_extraction/csvs/spec_encode_64_290_0.002042.pt.csv' does not exist: b'feature_extraction/csvs/spec_encode_64_290_0.002042.pt.csv'

In [11]:
ig=interactive_graph(df_64,grouping_by="drum_type",title="64 dim embedding")
display(ig.fig,widgets.HBox([ig.hover_data,ig.aud]))

ig=interactive_graph(df_64,grouping_by="label",title="64 dim embedding")
display(ig.fig,widgets.HBox([ig.hover_data,ig.aud]))

FigureWidget({
    'data': [{'customdata': array([['./dk_data/tom_high/Roland Tr-909-TR-909Tom Hi 01.wav'],
  …

FigureWidget({
    'data': [{'customdata': array([['./dk_data/tom_high/Roland Tr-909-TR-909Tom Hi 01.wav'],
  …

In [12]:
ig=interactive_graph(df_8,grouping_by="drum_type",title="8 dim embedding")
display(ig.fig,widgets.HBox([ig.hover_data,ig.aud]))

ig=interactive_graph(df_8,grouping_by="label",title="8 dim embedding")
display(ig.fig,widgets.HBox([ig.hover_data,ig.aud]))

FigureWidget({
    'data': [{'customdata': array([['./dk_data/tom_high/Roland Tr-909-TR-909Tom Hi 01.wav'],
  …

FigureWidget({
    'data': [{'customdata': array([['./dk_data/tom_high/Roland Tr-909-TR-909Tom Hi 01.wav'],
  …